### IMPORTS

In [1]:
import pandas as pd
import numpy as np
import pandas_gbq
import datetime
from dateutil.relativedelta import relativedelta
from gspread_pandas import Spread, conf

### CREDENCIALES

In [2]:
cred = conf.get_config('C:\\Users\\santiago.curat\\Pandas\\PEYA', 'PedidosYa-8b8c4d19f61c.json')

### QUERIES

In [3]:
q_coh = '''WITH barter_table AS (
    SELECT bt.trade.order_id AS order_id,
           bc.campaign_title AS campaign,
           bc.discount.type AS discount_type,
           bc.discount.amount AS discount_amount,
           bc.discount.max_amount AS discount_max_amount,
           bc.rules.min_amount AS min_amount           
    FROM `peya-data-origins-pro.cl_barter.trades` AS bt 
    LEFT JOIN `peya-data-origins-pro.cl_barter.creates` AS bc ON bt.id = bc.id
    WHERE DATE(bc.created_at) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND DATE_ADD(DATE_TRUNC(CURRENT_DATE(),MONTH), INTERVAL -1 DAY)
          AND DATE(bt.tradedAt) BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -4 MONTH),MONTH) AND DATE_ADD(DATE_TRUNC(CURRENT_DATE(),MONTH), INTERVAL -1 DAY)
          AND bt.trade.type = 'CONSUME'),
    orders_table AS (
    SELECT o.user.id AS user,
           cn.country_name AS country,
           IFNULL(c.city_name,'-') AS city,
           o.order_status AS status,
           o.order_id AS order_id,
           o.registered_date AS fecha,
           CASE WHEN bt.campaign IS NULL THEN 0 ELSE 1 END AS crm
    FROM `peya-bi-tools-pro.il_core.fact_orders` AS o
    LEFT JOIN barter_table AS bt ON o.order_id = bt.order_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_country` AS cn ON o.country_id = cn.country_id
    LEFT JOIN `peya-bi-tools-pro.il_core.dim_city` AS c ON o.city.city_id = c.city_id
    WHERE o.registered_date BETWEEN DATE_TRUNC(DATE_ADD(CURRENT_DATE(), INTERVAL -3 MONTH),MONTH) AND CURRENT_DATE()),
    acq_table AS (
    SELECT ot.user AS user,
           ot.country AS country,
           ot.city AS city,
           ot.fecha AS fecha,
           bt.campaign AS campaign,
           bt.discount_type AS discount_type,
           bt.discount_amount AS discount_amount,
           bt.discount_max_amount AS discount_max_amount,
           bt.min_amount AS min_amount
    FROM orders_table AS ot
    LEFT JOIN barter_table AS bt ON ot.order_id = bt.order_id
    WHERE bt.campaign IS NOT NULL),
    cohorts_table AS (
    SELECT acq.user AS user,
           acq.country AS country,
           acq.campaign AS campaign,
           acq.city AS city,
           acq.discount_type AS discount_type,
           acq.discount_amount AS discount_amount,
           acq.discount_max_amount AS discount_max_amount,
           acq.min_amount AS min_amount,
           acq.fecha AS fecha,
           CASE WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) = 0 THEN 'ORIGINALS'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 1 AND 30 THEN 'M1'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 31 AND 60 THEN 'M2'
                WHEN DATE_DIFF(ot.fecha,acq.fecha,DAY) BETWEEN 61 AND 90 THEN 'M3' END AS cohort,
           IFNULL(COUNT(DISTINCT CASE WHEN ot.status != 'CONFIRMED' THEN NULL ELSE ot.order_id END),0) AS orders
    FROM acq_table AS acq
    LEFT JOIN orders_table AS ot ON acq.user = ot.user
    WHERE ot.fecha BETWEEN acq.fecha AND DATE_ADD(acq.fecha, INTERVAL 180 DAY)
    GROUP BY 1,2,3,4,5,6,7,8,9,ot.fecha,ot.status),
    acq_final_table AS (
    SELECT FORMAT_DATE('%Y-%m', acq.fecha) AS fecha,
           acq.country AS country,
           acq.campaign AS campaign,
           acq.city AS city,
           acq.discount_type AS discount_type,
           acq.discount_amount AS discount_amount,
           acq.discount_max_amount AS discount_max_amount,
           acq.min_amount AS min_amount,
           COUNT(DISTINCT acq.user) AS users
    FROM acq_table AS acq
    GROUP BY 1,2,3,4,5,6,7,8)
SELECT FORMAT_DATE('%Y-%m', ct.fecha) AS Month,
       ct.country AS Country,
       ct.city AS City,
       ct.campaign AS Campaign,
       ct.discount_type AS Discount_Type,
       ct.discount_amount AS Amount,
       ct.discount_max_amount AS Max_Amount,
       ct.min_amount AS MOV,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'ORIGINALS' THEN ct.user ELSE NULL END) AS Acquisitions,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M1' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M1_Users,
       SUM(CASE WHEN ct.cohort = 'M1' THEN ct.orders ELSE 0 END) AS M1_orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M2' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M2_Users,
       SUM(CASE WHEN ct.cohort = 'M2' THEN ct.orders ELSE 0 END) AS M2_orders,
       COUNT(DISTINCT CASE WHEN ct.cohort = 'M3' AND ct.orders > 0 THEN ct.user ELSE NULL END) AS M3_Users,
       SUM(CASE WHEN ct.cohort = 'M3' THEN ct.orders ELSE 0 END) AS M3_orders
FROM cohorts_table AS ct
GROUP BY 1,2,3,4,5,6,7,8'''

In [149]:
# Descargo la data
bq_coh = pd.io.gbq.read_gbq(q_coh, project_id='peya-argentina', dialect='standard')

Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 193486/193486 [00:48<00:00, 3962.77rows/s]


In [4]:
# Copio las bases
coh = bq_coh.copy()

NameError: name 'bq_coh' is not defined

### TRABAJO

In [151]:
# Doy formato a las columnas
cols_str = ['Campaign','Country','Month','City','Discount_Type']
cols_float = [i for i in coh.columns if i not in cols_str]
coh[cols_float] = coh[cols_float].astype(float)
coh[cols_str] = coh[cols_str].apply(lambda x: x.astype(str).str.upper())
# Saco los espacios de los nombres de campañas
coh['Campaign'] = coh['Campaign'].str.replace(' ', '')
# Doy formato a la fecha
coh['Month'] = pd.to_datetime(coh['Month'], format='%Y-%m').dt.strftime('%Y-%m')
# Hago un listado de los paises
countries = list(coh['Country'].unique())
# Ordeno la base
coh.sort_values(by=['Country','Month','Campaign','City'],inplace=True)

In [152]:
# Creo las columnas faltantes
coh['Retention M1'] = coh['M1_Users'] / coh['Acquisitions']
coh['Retention M2'] = coh['M2_Users'] / coh['Acquisitions']
coh['Retention M3'] = coh['M3_Users'] / coh['Acquisitions']

In [153]:
cam = 'CRM_RMA_mix_mix_aut_CENTRAL_AR_Nacional_2103xx_RMA3-30_RMA'
index = ['Month','Country','Campaign']
values = ['Acquisitions','M1_Users','M1_orders','M2_Users','M2_orders','M3_Users','M3_orders','Retention M1','Retention M2','Retention M3']
coh[(coh['Country'] == 'ARGENTINA')&(coh['Campaign'] == cam.upper())&(coh['City'] == 'BUENOS AIRES')].pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

,Month,Country,Campaign,Acquisitions,M1_Users,M1_orders,M2_Users,M2_orders,M3_Users,M3_orders,Retention M1,Retention M2,Retention M3
0,2021-03,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,713,408,787,280,603,244,552,0.572230,0.392707,0.342216
1,2021-04,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,1811,1150,2626,718,1735,341,669,0.635008,0.396466,0.188294
2,2021-05,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,1577,904,2359,416,1220,0,0,0.573240,0.263792,0.000000


In [106]:
cam = 'CRM_RMA_mix_mix_aut_CENTRAL_AR_Nacional_2103xx_RMA3-30_RMA'
index = ['Month','Country','Campaign','Cohort']
values = ['Acquisitions','Orders','Active_Users']
coh[(coh['Country'] == 'ARGENTINA')&(coh['Campaign'] == cam.upper())].pivot_table(index=index,values=values,aggfunc='sum',fill_value=0).reset_index()

,Month,Country,Campaign,Cohort,Acquisitions,Active_Users,Orders
0,2021-03,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M1,8152,4812,8557
1,2021-03,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M2,8114,2738,5194
2,2021-03,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M3,7985,675,841
3,2021-03,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,ORIGINALS,8170,8170,9230
4,2021-04,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M1,24227,13775,25250
5,2021-04,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M2,24075,4590,7556
6,2021-04,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M3,2596,5,6
7,2021-04,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,ORIGINALS,24238,24238,28463
8,2021-05,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M1,18459,8203,14737
9,2021-05,ARGENTINA,CRM_RMA_MIX_MIX_AUT_CENTRAL_AR_NACIONAL_2103XX...,M2,4175,15,15
